# Cohere Multi-Step tool using Amazon Bedrock Converse API

---

#### Introduction
In this notebook, we demonstrate how to use the Bedrock Converse API with the `list_calendar_events` and `create_calendar_event` tools to book appointments. Booking the correct appointment requires the model to first check for an available slot by listing existing events, reasoning about the correct slot to book the new appointment and then finally invoking the right tool to create the calendar event. All code was orinally provided by Cohere and this notebook is a supplement of the original notebook but now instead uses Amazon Bedrock's Converse API.

---

## Amazon Bedrock

Amazon Bedrock is a fully managed service that provides access to a wide range of powerful foundational models(FMs) through an API.

## Bedrock Converse API

The AWS Bedrock Converse API provides a consistent interface for sending and receiving messages to and from different large language models (LLMs) on the Bedrock platform. It allows developers to write code once and use it with various models, making it easier to switch between them. The Converse API simplifies tool use (function calling) with LLMs by providing a standardized way to define and invoke tools across different models. It eliminates the need for model-specific tool processing or structure. With the Converse API, developers can define tools using a JSON schema, and the LLM can generate JSON output conforming to that schema, effectively invoking the tool without an actual function call. 

By abstracting away the differences in tool use conventions across models, the Converse API makes it easier to integrate and utilize tools with LLMs on the Bedrock platform. This improves the success rate of agent tasks and reduces hallucinations, as models like Cohere have specific tool use APIs that enhance their performance.

## Getting Started

### Step 0: Install Dependencies

Here, we will install all the required dependencies to run this notebook.



In [117]:
%pip install boto3 IPython --quiet

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


#### Now lets import the required modules to run the notbook

In [118]:
# Instantiate the Bedrock client
import boto3
import json
bedrock_rt = boto3.client(service_name="bedrock-runtime")

### Step 1: Select Model

Below sets default model to command R+ model with option to change to Command R

In [119]:
#define command R and command R+ models
DEFAULT_MODEL= "cohere.command-r-plus-v1:0"
COMMAND_R_PLUS = "cohere.command-r-plus-v1:0"
COMMAND_R = "cohere.command-r-v1:0"
model_id = DEFAULT_MODEL

### Step 2: Create tool functions

Below code creates two functions for each tool we want to use. We define functions for the list_calendar_events as well as create_calendar_event

In [120]:
# Define the tools

def list_calendar_events(date: str):
    events =  '[{"start": "14:00", "end": "15:00"}, {"start": "15:00", "end": "16:00"}, {"start": "17:00", "end": "18:00"}]'
    print(f"Listing events: {events}")
    return events

def create_calendar_event(date: str, time: str, duration: int):
  print(f"Creating a {duration} hour long event at {time} on {date}")
  return True
  

### Step 3: Define the tools

Next, we define the json to define the actual tools to then send into Bedrock. This is the standard structure to use for tool use (function calling) with Cohere models with Bedrock Converse API.

We set the description to a value that accurately explains what we want the tool to do. It is important to be clear, concise for actions the tool will do. This is because the model will leverage these descriptions when selecting the tools to answer customers query or fufil the input request.

In [121]:

list_calendar_events_tool = {
        'name': 'list_calendar_events',
        'description': 'returns a list of calendar events for the specified date, including the start time and end time for each event',
        'inputSchema': {
            'json': {
                'type': 'object',
                'properties': {
                    'date': {
                        'type': 'string',
                        'description': 'the date to list events for, formatted as mm/dd/yy'
                    }
                },
                'required': [
                    'date'
                ]
            }
        }
    }


create_calendar_event_tool = {
        'name': 'create_calendar_event_tool',
        'description': 'creates a calendar event of the specified duration at the specified time and date',
        'inputSchema': {
            'json': {
                'type': 'object',
                'properties': {
                    'date': {
                        'type': 'string',
                        'description': 'the date to list events for, formatted as mm/dd/yy'
                    },
                    'time': {
                        'type': 'string',
                        'description': 'the time of the event, formatted using 24h military time formatting'
                    },
                    'duration': {
                        'type': 'float',
                        'description': 'the number of hours the event lasts for'
                    }
                    
                },
            
                'required': [
                    'date', 'time', 'duration'
                ]
            }
              
            }
        }

## Step 4: Tool Configuration for Bedrock Converse API

The `tool_config` variable is a crucial component when setting up the Bedrock Converse API. It defines the configuration for the tools that the llm can use during conversations.

Here is an example of the sructure of `tool_config`

```python
tool_config = {
    "type": "structured_tool",
    "function": {
        "name": "get_current_weather",
        "description": "Get the current weather in a given location",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city and state, e.g. San Francisco, CA"
                },
                "unit": {
                    "type": "string",
                    "enum": ["celsius", "fahrenheit"]
                }
            },
            "required": ["location"]
        }
    }
}


We will create the tool_config by appending each tool defined above. 

In [122]:
tool_config = {'tools': []}

tool_config['tools'].append({
    'toolSpec': list_calendar_events_tool
})


tool_config['tools'].append({
    'toolSpec': create_calendar_event_tool
               
})

## Step 5: Define invoke_tool function to call the right tool

We define this function to help us identify which tool the llm shoud used once the name of the tool is identied. The Converse API will output a tool block including the name of the tool the lllm identied should be used. 


In [127]:
def invoke_tool(tool_call):
  # Iterate over the tool calls generated by the model
    if tool_call['name'] == 'list_calendar_events':
        date = tool_call['input']['date']
        return [{
             "events": list_calendar_events(date)
          }]
    elif tool_call['name'] == 'create_calendar_event_tool':
        date = tool_call['input']['date']
        time = tool_call['input']['time']
        duration = tool_call['input']['duration']
        return [{
             "is success": create_calendar_event(date, time, duration)
           }]
    else:
        raise f"Unnown tool name '{tool_call['name']}'"

Now we call the Converse API and return the output response. Append each response to the list of messages to keep the standardized format for Bedrock APIs

In [131]:
# Check what tools the model wants to use and how to use them
additional_model_fields ={"preamble": """
## Task & Context
You help people answer their questions and other requests interactively. You will be asked a very wide array of requests on all kinds of topics. You will be equipped with a wide range of search engines or similar tools to help you, which you use to research your answer. You should focus on serving the user's needs as best you can, which will be wide-ranging.

## Style Guide
Unless the user asks for a different style of answer, you should answer in full sentences, using proper grammar and spelling.
                          
## Current Date
Today is Thursday, may 23, 2024                      
"""
}

#additional_model_fields ={"preamble": "Today is Thursday, may 23, 2024"}
input_text = "can you check when the first available free slot after 3pm is and then make an hour long appointment for that day"
def converse(messages, tool_config,  model_id):
    response = bedrock_rt.converse(
        messages=messages,
        toolConfig=tool_config,
        additionalModelRequestFields=additional_model_fields,
        modelId = model_id
    )
    return response

#create an initial message to pass the input text into. This is standard format for messages parameter in Converse API
messages =[]
messages.append({
        "role": "user",
        "content": [{"text": input_text}]
    }
)

response = converse(messages, tool_config, model_id)

response_message = response['output']['message']
#we want to pass this response_message back in
response_content = response_message['content']
messages.append(response_message)

print("The model recommends doing the following tool calls:")
for content in response['output']['message']['content']:
    if 'toolUse' in content:
        print("\n")
        print(content['toolUse'])

The model recommends doing the following tool calls:


{'toolUseId': 'tooluse_0TM_DMilTka0vBAOph1WSw', 'name': 'list_calendar_events', 'input': {'date': '05/23/2024'}}


## Step 6: Tool Invocation and Response Handling in Bedrock Converse API

This code snippet demonstrates how to handle tool invocations and process responses in the context of the Bedrock Converse API. It's a crucial part of implementing a conversational AI system that can use external tools to enhance its capabilities.

In [133]:
follow_up_message = []

for content in response_message['content']:
    if 'toolUse' in content:

        tool_use_block = content['toolUse']
        tool_use_name = tool_use_block['name']
        tool_result_value = invoke_tool(tool_use_block)
        follow_up_message.append({
            "toolResult": {
                "toolUseId": tool_use_block['toolUseId'],
                "content": [{"json": {"result": tool_result_value}}]
            }
        })
print(response_content[0]['text'])

if follow_up_message:
    follow_up_message = {"role": "user", "content": follow_up_message}
    messages.append(follow_up_message)
    response = bedrock_rt.converse(
        modelId=model_id,
        messages=messages,
        toolConfig=tool_config
    )
    response_message = response['output']['message']
    messages.append(response_message)
print(response_content[0]['text'])

Creating a 1 hour long event at 18:00 on 05/23/2024
I will first check the user's calendar for 23 May 2024 after 3pm to find the first available slot. Then, I will create an hour-long appointment for that time.
I will first check the user's calendar for 23 May 2024 after 3pm to find the first available slot. Then, I will create an hour-long appointment for that time.


#### Conclusion
This notebook follows the official Cohere Cookbooks and converts the original Cohere cookbook for multi tool use into a notebook that uses the Bedrock Converse API. We explored multi-tool use as well as bring in conversation history from an initial output into a second turn in the conversation